# Text-Conv Example with PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# from softmax import Softmax

### Define CNN model network

In [2]:
class CnnModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim=24):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.conv1 = nn.Conv1d(in_channels=16, out_channels=128, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=256, kernel_size=4)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=512, kernel_size=5)
        
        # flatten and concat conv1, conv2, conv3
        self.fc1 = nn.Linear(in_features=(128+256+512), out_features = 256)
        self.fc2 = nn.Linear(in_features=256, out_features=2)
        
    def forward(self, t):
        # (0) input layer
        t = t
        
        # (1) embedding layer (this layer assign a vector to each word index in t)
        t = self.embeddings(t)
        
        # (2) hidden conv layer
        tri_gram = self.conv1(t)
        tri_gram = F.relu(tri_gram)
        tri_gram = F.max_pool1d(tri_gram, kernel_size=tri_gram.shape[2], stride=1) # GlobalMaxPool1d using MaxPool1d
        
        # (3) hidden conv layer
        four_gram = self.conv2(t)
        four_gram = F.relu(four_gram)
        four_gram = F.max_pool1d(four_gram, kernel_size=four_gram.shape[2], stride=1) # GlobalMaxPool1d using MaxPool1d
        
        # (4) hidden conv layer
        five_gram = self.conv3(t)
        five_gram = F.relu(five_gram)
        five_gram = F.max_pool1d(five_gram, kernel_size=five_gram.shape[2], stride=1) # GlobalMaxPool1d using MaxPool1d
        
        # flatten and concat conv1, conv2, conv3
        t = torch.cat((tri_gram, four_gram, five_gram), dim=1).squeeze(dim=2)
        
        # (6) hidden linear layer
        t = self.fc1(t)
        t = F.relu(t)
        
        # (7) output layer
        #t = self.out(t)
        #t = F.softmax(t, dim=1)
        
        return t

### Creat CNN model

In [10]:
cnn_model = CnnModel(vocab_size=1000)
cnn_model

CnnModel(
  (embeddings): Embedding(1000, 24, padding_idx=0)
  (conv1): Conv1d(16, 128, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(16, 256, kernel_size=(4,), stride=(1,))
  (conv3): Conv1d(16, 512, kernel_size=(5,), stride=(1,))
  (fc1): Linear(in_features=896, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)

### Creat random data

In [5]:
# Creat random word indices in the vocabulary
data = torch.randint(0, 1000, (1, 16)) # arguments: start_range=0, end_range=1000, (batch_size, words_in_instance=16)
data.shape

torch.Size([1, 16])

In [9]:
data

tensor([[373, 542, 433, 632, 208, 321,   2, 917, 772, 772, 788,  54, 434, 864,
         506, 909]])

### Forward pass

In [6]:
out = cnn_model(data)
out.shape

C:\Users\basharm\Anaconda3\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([1, 256])

In [7]:
out[0][127]

tensor(0.3469, grad_fn=<SelectBackward>)